In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

In [17]:
test = pd.read_csv('../output/merge_data.csv')
test.head()

,click_mode,pid,sid,o1,o2,d1,d2,weekday,hour,time_index,...,profile_embedding_0,profile_embedding_1,profile_embedding_2,profile_embedding_3,profile_embedding_4,profile_embedding_5,profile_embedding_6,profile_embedding_7,profile_embedding_8,profile_embedding_9
0,9.0,-2.0,3000821,116.29,39.97,116.32,39.96,4,17,107,...,0.039524,0.018515,-0.000454,0.000419,-0.001196,-0.000105,0.000206,0.000705,-0.000817,0.000203
1,1.0,210736.0,3085857,116.39,39.84,116.33,39.79,4,10,65,...,0.000429,0.001080,-0.000958,-0.001472,0.005630,0.001093,0.007163,0.004830,-0.004908,-0.006151
2,9.0,-2.0,2944522,116.31,39.93,116.27,40.00,5,10,63,...,0.039524,0.018515,-0.000454,0.000419,-0.001196,-0.000105,0.000206,0.000705,-0.000817,0.000203
3,1.0,202427.0,559931,116.27,39.88,116.39,39.90,4,14,89,...,0.005781,-0.013828,-0.000806,-0.001470,0.000740,-0.002083,0.002893,-0.007920,-0.010410,-0.001802
4,7.0,172251.0,2819352,116.34,39.96,116.37,39.86,1,11,70,...,0.000715,0.000271,0.003487,-0.004889,-0.002707,-0.007278,-0.001466,0.000578,0.006275,0.003834


In [19]:
cate_cols = ['max_dist_mode', 'min_dist_mode', 'max_price_mode',
             'min_price_mode', 'max_eta_mode', 'min_eta_mode', 'first_mode', 'last_mode','weekday', 'hour', 'weather'
             ]

for col in cate_cols:
    print (test[col].dtype, test[col].min(), test[col].unique())

float64 -1.0 [ 2.  7.  1.  4.  9.  3. 10.  6. 11. -1.  5.  8.]
float64 -1.0 [ 5.  3.  6.  1.  2.  7. -1. 10.  4.  9. 11.  8.]
float64 -1.0 [ 4.  1.  3.  6.  7.  2. -1. 10.  8.  9.  5. 11.]
float64 -1.0 [ 5.  1.  9.  3.  7.  6.  2. 11. -1.  4.]
float64 -1.0 [ 5.  1.  7.  6. 10.  4.  3.  2. -1.  9. 11.  8.]
float64 -1.0 [ 3.  6.  2. 10.  9. -1.  4.  7.  5.  1. 11.]
float64 -1.0 [ 9.  7. 10.  3.  1.  5.  6.  4.  2. 11. -1.  8.]
float64 -1.0 [ 5.  1.  7.  4.  9.  6.  3. 10.  2. -1. 11.  8.]
int64 0 [4 5 1 3 2 0 6]
int64 0 [17 10 14 11  7  9 15 23  1 16 13 12 21  8  0 18  6 19 20  2  5 22  4  3]
int64 0 [0 1 5 2 3 4]


In [26]:
# check the missing data in cate cols
x_train = pd.read_csv('../tmp/train_x.csv')
x_test = pd.read_csv('../tmp/test_x.csv')
x_train.head()

,o1,o2,d1,d2,weekday,hour,time_index,max_temp,min_temp,weather,...,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11
0,116.29,39.97,116.32,39.96,4,17,107,18,4,0,...,0.009918,0.023780,0.019775,0.015374,0.018348,0.000352,0.000485,0.001259,-0.001219,0.000967
1,116.39,39.84,116.33,39.79,4,10,65,8,-1,0,...,-0.003237,0.001644,0.000313,-0.001904,0.001617,0.000233,0.000178,-0.000263,0.000527,-0.001440
2,116.31,39.93,116.27,40.00,5,10,63,20,8,0,...,0.009918,0.023780,0.019775,0.015374,0.018348,0.000352,0.000485,0.001259,-0.001219,0.000967
3,116.27,39.88,116.39,39.90,4,14,89,7,0,1,...,0.017296,0.011147,-0.010151,-0.004170,-0.004494,0.014182,0.000904,-0.007829,0.006432,0.004281
4,116.34,39.96,116.37,39.86,1,11,70,16,1,0,...,-0.004666,-0.006974,0.003199,-0.006893,0.011231,0.001814,-0.001417,0.003012,0.004929,-0.002483


In [27]:
for col in cate_cols:
    print(x_train[col].dtype, x_train[col].min(), x_train[col].unique())
    
print('for test', '\n')

for col in cate_cols:
    print(x_test[col].dtype, x_test[col].min(), x_test[col].unique())

int64 -1 [ 2  7  1  4  9  3 10  6 11 -1  5  8]
int64 -1 [ 5  3  6  1  2  7 -1 10  4  9 11  8]
int64 -1 [ 4  1  3  6  7  2 -1 10  8  9  5 11]
int64 -1 [ 5  1  9  3  7  6  2 11 -1  4  8 10]
int64 -1 [ 5  1  7  6 10  4  3  2 -1  9 11  8]
int64 -1 [ 3  6  2 10  9 -1  4  7  5  1 11  8]
int64 -1 [ 9  7 10  3  1  5  6  4  2 11 -1  8]
int64 -1 [ 5  1  7  4  9  6  3 10  2 -1 11  8]
int64 0 [4 5 1 3 2 0 6]
int64 0 [17 10 14 11  7  9 15 23  1 16 13 12 21  8  0 18  6 19 20  2  5 22  4  3]
int64 0 [0 1 5 2 3 4]
for test 

int64 -1 [ 2  1  7  3 11  8  9  6 10  4 -1  5]
int64 -1 [ 3  6  5  1  4  2  7  9  8 11 -1 10]
int64 -1 [ 4  1  3  6 -1  7 10 11  2  5  8  9]
int64 -1 [ 2  6  1  5  9  7  3 -1 11  4  8]
int64 -1 [ 7  5  1 11  2  4  3  8 10  6 -1  9]
int64 -1 [ 3  6  4  2  9  5 -1 10  7  1 11  8]
int64 -1 [ 2  1  7  4  5  3 10 -1  6  8  9 11]
int64 -1 [ 7  5  1 11  3  4  8  9 10  2  6 -1]
int64 0 [2 5 4 3 1 0 6]
int64 0 [14 12 10  7 20 15 13 11 17 19 18 22  9  2 16  8 23 21  5  6  4  0  1  3]
int64 

In [39]:
weather = pd.read_json('../data/weather.json')
weather.head()



,10-01,10-02,10-03,10-04,10-05,10-06,10-07,10-08,10-09,10-10,...,11-30,12-01,12-02,12-03,12-04,12-05,12-06,12-07,12-08,12-09
max_temp,24,24,25,25,24,20,21,21,15,17,...,8,7,9,8,4,1,-2,-4,-2,-1
min_temp,12,11,10,12,14,8,7,8,4,4,...,-3,0,2,-3,-6,-4,-9,-10,-10,-10
weather,q,q,q,q,dy,q,q,dy,dyq,dyq,...,q,dy,dy,dyq,qdy,dy,q,q,qdy,dyq
wind,45,12,12,12,12,45,12,12,45,12,...,1,1,1,3,2,1,3,3,2,1


In [71]:
df_weather = weather.T.reset_index()
df_weather.columns

Index(['index', 'max_temp', 'min_temp', 'weather', 'wind'], dtype='object')

In [72]:
df_weather.columns = ['date', 'max_temp', 'min_temp', 'weather', 'wind']
df_weather.head()

,date,max_temp,min_temp,weather,wind
0,10-01,24,12,q,45
1,10-02,24,11,q,12
2,10-03,25,10,q,12
3,10-04,25,12,q,12
4,10-05,24,14,dy,12


In [65]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 5 columns):
date        70 non-null object
max_temp    70 non-null object
min_temp    70 non-null object
weather     70 non-null object
wind        70 non-null object
dtypes: object(5)
memory usage: 2.8+ KB


In [73]:
df_weather['weather'].unique()
weather_dict = {}
for i, item in enumerate(df_weather['weather'].unique()):
    weather_dict[item]=i
df_weather['weather'] = df_weather['weather'].replace(weather_dict)


In [74]:
df_weather['date'] = '2018-' + df_weather['date']
df_weather['date'].unique()

array(['2018-10-01', '2018-10-02', '2018-10-03', '2018-10-04',
       '2018-10-05', '2018-10-06', '2018-10-07', '2018-10-08',
       '2018-10-09', '2018-10-10', '2018-10-11', '2018-10-12',
       '2018-10-13', '2018-10-14', '2018-10-15', '2018-10-16',
       '2018-10-17', '2018-10-18', '2018-10-19', '2018-10-20',
       '2018-10-21', '2018-10-22', '2018-10-23', '2018-10-24',
       '2018-10-25', '2018-10-26', '2018-10-27', '2018-10-28',
       '2018-10-29', '2018-10-30', '2018-10-31', '2018-11-01',
       '2018-11-02', '2018-11-03', '2018-11-04', '2018-11-05',
       '2018-11-06', '2018-11-07', '2018-11-08', '2018-11-09',
       '2018-11-10', '2018-11-11', '2018-11-12', '2018-11-13',
       '2018-11-14', '2018-11-15', '2018-11-16', '2018-11-17',
       '2018-11-18', '2018-11-19', '2018-11-20', '2018-11-21',
       '2018-11-22', '2018-11-23', '2018-11-24', '2018-11-25',
       '2018-11-26', '2018-11-27', '2018-11-28', '2018-11-29',
       '2018-11-30', '2018-12-01', '2018-12-02', '2018-

In [75]:
df_weather['date'] = pd.to_datetime(df_weather['date'], format = '%Y-%m-%d')
df_weather[['max_temp', 'min_temp', 'weather','wind']] = df_weather[['max_temp', 'min_temp', 'weather','wind']].astype(int)
df_weather.head()

,date,max_temp,min_temp,weather,wind
0,2018-10-01,24,12,0,45
1,2018-10-02,24,11,0,12
2,2018-10-03,25,10,0,12
3,2018-10-04,25,12,0,12
4,2018-10-05,24,14,1,12


In [76]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 5 columns):
date        70 non-null datetime64[ns]
max_temp    70 non-null int64
min_temp    70 non-null int64
weather     70 non-null int64
wind        70 non-null int64
dtypes: datetime64[ns](1), int64(4)
memory usage: 2.8 KB


In [36]:
df_test = pd.read_csv('../output/test.csv')
df_test.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,5.2,6.2,7.2,8.2,9.2,10.2,11.2,mode,before_pred_mode,after_pred_mode
0,0,0.068880,0.000426,0.078395,0.043649,0.018417,0.005002,0.019279,0.000389,0.000015,...,0.005002,0.019279,0.000000,0.0,0.765448,0.0,0.0,9.0,9,9
1,1,0.155122,0.146756,0.000497,0.101397,0.077253,0.000214,0.000083,0.326504,0.000025,...,0.000000,0.000000,0.326504,0.0,0.191938,0.0,0.0,1.0,7,7
2,2,0.095294,0.000343,0.000504,0.065693,0.065109,0.000463,0.000147,0.771957,0.000026,...,0.000000,0.000000,0.771957,0.0,0.000000,0.0,0.0,7.0,7,7
3,3,0.471206,0.288266,0.000660,0.143442,0.094596,0.000552,0.000246,0.000473,0.000029,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.0,0,1
4,4,0.071528,0.782407,0.000404,0.046757,0.016675,0.023404,0.058224,0.000276,0.000020,...,0.023404,0.058224,0.000000,0.0,0.000000,0.0,0.0,1.0,1,1


In [37]:
df_filter = df_test.query('before_pred_mode!=after_pred_mode')
df_filter.to_csv('../output/test_filter.csv')

In [ ]:
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)
# avail_cols = ['mode_avail_{}'.format(i) for i in range(12)]
# cv_count = 0
# for tr_idx, val_idx in kfold.split(train_x, train_y):
#     print(val_idx[:2])
#     print(train_y.head())
#     if cv_count<1:
#         val_y = train_y[[0,1]]

In [77]:
tr_queries = pd.read_csv('../data/train_queries.csv')

tr_plans = pd.read_csv('../data/train_plans.csv')


tr_click = pd.read_csv('../data/train_clicks.csv')

tr_data = tr_queries.merge(tr_click, on='sid', how='left')
tr_data = tr_data.merge(tr_plans, on='sid', how='left')
tr_data = tr_data.drop(['click_time'], axis=1)
tr_data['click_mode'] = tr_data['click_mode']

In [78]:
tr_data['req_time']=pd.to_datetime(tr_data['req_time'])


In [82]:
tr_data['date'] = tr_data['req_time'].dt.normalize()
tr_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 9 columns):
sid           500000 non-null int64
pid           336021 non-null float64
req_time      500000 non-null datetime64[ns]
o             500000 non-null object
d             500000 non-null object
click_mode    453336 non-null float64
plan_time     491054 non-null object
plans         491054 non-null object
date          500000 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(4)
memory usage: 58.1+ MB


In [83]:
tr_data.head()

,sid,pid,req_time,o,d,click_mode,plan_time,plans,date
0,3000821,NaN,2018-11-02 17:54:30,"116.29,39.97","116.32,39.96",9.0,2018-11-02 17:54:30,"[{""distance"": 5219, ""price"": 300, ""eta"": 1367,...",2018-11-02
1,3085857,210736.0,2018-11-16 10:53:10,"116.39,39.84","116.33,39.79",1.0,2018-11-16 10:53:10,"[{""distance"": 13864, ""price"": 600, ""eta"": 3221...",2018-11-16
2,2944522,NaN,2018-10-06 10:33:58,"116.31,39.93","116.27,40.00",9.0,2018-10-06 10:33:58,"[{""distance"": 12294, ""price"": 400, ""eta"": 2472...",2018-10-06
3,559931,202427.0,2018-11-23 14:54:11,"116.27,39.88","116.39,39.90",1.0,2018-11-23 14:54:11,"[{""distance"": 14853, ""price"": 1700, ""eta"": 291...",2018-11-23
4,2819352,172251.0,2018-10-30 11:48:41,"116.34,39.96","116.37,39.86",7.0,2018-10-30 11:48:41,"[{""distance"": 12882, ""price"": 600, ""eta"": 3211...",2018-10-30


In [112]:
tr_plans.query('sid == 951150|sid==2085587')

,sid,plan_time,plans
78127,951150,2018-10-24 13:18:37,"[{""distance"": 40320, ""price"": 900, ""eta"": 6463..."


In [113]:
tr_plans.query('sid == 951150| sid ==2085587')['plans'].values

array(['[{"distance": 40320, "price": 900, "eta": 6463, "transport_mode": 11}, {"distance": 39345, "price": "", "eta": 4083, "transport_mode": 3}, {"distance": 39345, "price": 12100, "eta": 4083, "transport_mode": 4}, {"distance": 40320, "price": 900, "eta": 7036, "transport_mode": 7}, {"distance": 41004, "price": 1000, "eta": 9158, "transport_mode": 1}]'],
      dtype=object)

In [114]:
tr_click.query('sid==951150|sid==2085587')

,sid,click_time,click_mode


In [73]:
print(train_x.shape)
print(train_y.shape)

(500000, 67)
(500000, 1)


In [94]:
list(train_y.loc[:100,:].to_dict()['train_y'].values())[:20]

[9.0,
 1.0,
 9.0,
 1.0,
 7.0,
 1.0,
 1.0,
 1.0,
 5.0,
 3.0,
 1.0,
 1.0,
 9.0,
 1.0,
 1.0,
 1.0,
 2.0,
 5.0,
 9.0,
 2.0]

In [95]:
np.array(list(train_y.loc[:100,:].to_dict()['train_y'].values()))[:20]

array([9., 1., 9., 1., 7., 1., 1., 1., 5., 3., 1., 1., 9., 1., 1., 1., 2.,
       5., 9., 2.])

In [10]:
tmp.columns

Index(['Unnamed: 0', 'o1', 'o2', 'd1', 'd2', 'mode_feas_0', 'mode_feas_1',
       'mode_feas_2', 'mode_feas_3', 'mode_feas_4', 'mode_feas_5',
       'mode_feas_6', 'mode_feas_7', 'mode_feas_8', 'mode_feas_9',
       'mode_feas_10', 'mode_feas_11', 'max_dist', 'min_dist', 'mean_dist',
       'std_dist', 'max_price', 'min_price', 'mean_price', 'std_price',
       'max_eta', 'min_eta', 'mean_eta', 'std_eta', 'max_dist_mode',
       'min_dist_mode', 'max_price_mode', 'min_price_mode', 'max_eta_mode',
       'min_eta_mode', 'first_mode', 'svd_mode_0', 'svd_mode_1', 'svd_mode_2',
       'svd_mode_3', 'svd_mode_4', 'svd_mode_5', 'svd_mode_6', 'svd_mode_7',
       'svd_mode_8', 'svd_mode_9', 'svd_fea_0', 'svd_fea_1', 'svd_fea_2',
       'svd_fea_3', 'svd_fea_4', 'svd_fea_5', 'svd_fea_6', 'svd_fea_7',
       'svd_fea_8', 'svd_fea_9', 'svd_fea_10', 'svd_fea_11', 'svd_fea_12',
       'svd_fea_13', 'svd_fea_14', 'svd_fea_15', 'svd_fea_16', 'svd_fea_17',
       'svd_fea_18', 'svd_fea_19', 'weekday'

In [12]:
# tmp.loc[:1000,:].to_csv('../tmp/tmp_sample.csv', index=False)

In [70]:
# check how sklearn computes tf_idf matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
corpus = [
    'This is the first document.',
    'This documentis the second document.',
    'And this is the third one.',
    'Is this the first document?'
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)


vectorizer_count = CountVectorizer()
X_count = vectorizer_count.fit_transform(corpus)

# vectorizer2 = TfidfTransformer()
# X_ = vectorizer2.fit_transform(corpus)

In [71]:
print(X)
print(X_count)


  (0, 9)	0.37075825825799347
  (0, 4)	0.45349057268475307
  (0, 7)	0.37075825825799347
  (0, 3)	0.5601510767990253
  (0, 1)	0.45349057268475307
  (1, 9)	0.30372247895185167
  (1, 7)	0.30372247895185167
  (1, 1)	0.3714961915191231
  (1, 2)	0.5820205714772765
  (1, 6)	0.5820205714772765
  (2, 9)	0.26249892378623274
  (2, 4)	0.3210738658560508
  (2, 7)	0.26249892378623274
  (2, 0)	0.503024254778498
  (2, 8)	0.503024254778498
  (2, 5)	0.503024254778498
  (3, 9)	0.37075825825799347
  (3, 4)	0.45349057268475307
  (3, 7)	0.37075825825799347
  (3, 3)	0.5601510767990253
  (3, 1)	0.45349057268475307
  (0, 1)	1
  (0, 3)	1
  (0, 7)	1
  (0, 4)	1
  (0, 9)	1
  (1, 6)	1
  (1, 2)	1
  (1, 1)	1
  (1, 7)	1
  (1, 9)	1
  (2, 5)	1
  (2, 8)	1
  (2, 0)	1
  (2, 7)	1
  (2, 4)	1
  (2, 9)	1
  (3, 1)	1
  (3, 3)	1
  (3, 7)	1
  (3, 4)	1
  (3, 9)	1


In [65]:
print(X_count[:,1].sum())

3


In [116]:
X.shape

(4, 10)

In [28]:
import numpy as np
X = np.array([[1,1,1,0,0],[3,3,3,0,0],[4,4,4,0,0],[5,5,5,0,0],[0,0,0,4,4],[0,0,0,5,5],[0,0,0,2,2]])


In [29]:
X

array([[1, 1, 1, 0, 0],
       [3, 3, 3, 0, 0],
       [4, 4, 4, 0, 0],
       [5, 5, 5, 0, 0],
       [0, 0, 0, 4, 4],
       [0, 0, 0, 5, 5],
       [0, 0, 0, 2, 2]])

In [30]:
from sklearn.decomposition import TruncatedSVD
svd_enc = TruncatedSVD(n_components=2)
mode_svd = svd_enc.fit_transform(X)

In [31]:
mode_svd

array([[ 1.73205081e+00, -9.80146262e-16],
       [ 5.19615242e+00,  2.83343628e-15],
       [ 6.92820323e+00, -1.12251545e-15],
       [ 8.66025404e+00, -1.77236652e-15],
       [ 1.84079772e-15,  5.65685425e+00],
       [ 2.34974370e-15,  7.07106781e+00],
       [ 9.21383665e-16,  2.82842712e+00]])

In [32]:
np.dot(X, svd_enc.components_.T)

array([[ 1.73205081e+00, -2.19856920e-16],
       [ 5.19615242e+00, -6.59570760e-16],
       [ 6.92820323e+00, -8.79427680e-16],
       [ 8.66025404e+00, -1.09928460e-15],
       [ 7.18049694e-16,  5.65685425e+00],
       [ 8.97562117e-16,  7.07106781e+00],
       [ 3.59024847e-16,  2.82842712e+00]])

In [37]:
tmp = np.dot( svd_enc.components_.T, svd_enc.components_).round(3)
tmp

array([[ 0.333,  0.333,  0.333,  0.   ,  0.   ],
       [ 0.333,  0.333,  0.333, -0.   , -0.   ],
       [ 0.333,  0.333,  0.333,  0.   ,  0.   ],
       [ 0.   , -0.   ,  0.   ,  0.5  ,  0.5  ],
       [ 0.   , -0.   ,  0.   ,  0.5  ,  0.5  ]])

In [38]:
np.dot(X, tmp)

array([[0.999, 0.999, 0.999, 0.   , 0.   ],
       [2.997, 2.997, 2.997, 0.   , 0.   ],
       [3.996, 3.996, 3.996, 0.   , 0.   ],
       [4.995, 4.995, 4.995, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 4.   , 4.   ],
       [0.   , 0.   , 0.   , 5.   , 5.   ],
       [0.   , 0.   , 0.   , 2.   , 2.   ]])

In [120]:
print(svd_enc.components_)
print(svd_enc.singular_values_)

[[ 5.77350269e-01  5.77350269e-01  5.77350269e-01 -4.38262753e-21
  -4.38262753e-21]
 [ 3.27845560e-21  5.95170575e-21  1.50503983e-21  7.07106781e-01
   7.07106781e-01]]
[12.36931688  9.48683298]


In [128]:
svd_enc_T = TruncatedSVD(n_components=2, n_iter=20, random_state=2019)
mode_svd_T = svd_enc_T.fit_transform(X.T)
print(svd_enc_T.components_)
print(svd_enc_T.singular_values_)

[[ 1.40028008e-01  4.20084025e-01  5.60112034e-01  7.00140042e-01
  -7.56199941e-18  7.54724459e-18 -3.78032375e-18]
 [-8.93658912e-17  4.00855628e-17 -2.74634218e-17  1.58125567e-17
   5.96284794e-01  7.45355992e-01  2.98142397e-01]]
[12.36931688  9.48683298]


In [41]:
print(vectorizer.vocabulary_)

{'and': 0, 'is': 4, 'third': 8, 'this': 9, 'first': 3, 'second': 6, 'document': 1, 'the': 7, 'one': 5, 'documentis': 2}


In [42]:
print(vectorizer.get_feature_names())

['and', 'document', 'documentis', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [8]:
def f1_decomposition(val_y, val_pred):
    precision, recall, F1, support = precision_recall_fscore_support(val_y, val_pred)
    weighted_F1 =  precision_recall_fscore_support(val_y, val_pred, average ='weighted')[2]
    df_eval = pd.DataFrame({'precision':precision, 'recall':recall,'F1':F1, 'support':support, 'weighted_F1':weighted_F1})
    return df_eval


def augment_minority_class(val_proba, val_y):

    val_score_list = []
    f1_decomposition_list=[]
    search_len = 20
    for weight1 in range(search_len):
        for weight2 in range(search_len):
            val_proba_tmp = val_proba.copy()
            weight1 = weight1/10.0 + 1
            weight2 = weight2/10.0 + 1
            val_proba_tmp[:,3]=val_proba_tmp[:,3]*weight1
            val_proba_tmp[:,4]=val_proba_tmp[:,4]*weight2
            val_pred_tmp = np.argmax(val_proba_tmp, axis=1)
            val_score = f1_score(val_y, val_pred_tmp, average='weighted')
            df_f1_decomposition = f1_decomposition(val_y, val_pred_tmp)
            val_score_list.append(val_score)
            f1_decomposition_list.append(df_f1_decomposition)
    max_index = np.argmax(np.array(val_score_list))
    print(val_score_list)
    print("weight1:", max_index//search_len, "weight2:", max_index%search_len)
    print(f1_decomposition_list[max_index])




In [9]:
#grid search for weights
val_proba = pd.read_csv("../tmp/val_proba.csv").values
val_y = pd.read_csv("../tmp/val_y.csv").values
augment_minority_class(val_proba, val_y)

[0.6944785166650189, 0.6951928749599697, 0.6953567096251628, 0.6954815254227094, 0.6957280001724162, 0.6958432338827358, 0.6960023065289213, 0.6960505427887894, 0.6961930430561075, 0.6964094843425619, 0.6964652327220255, 0.6966813731216281, 0.6967657623610637, 0.6968132249037837, 0.6967451842025046, 0.6967467076228651, 0.6968151229760088, 0.6968750974601067, 0.6969656534939728, 0.6968722118162736, 0.6949598228531916, 0.6951662766343041, 0.6953567096251628, 0.6954815254227094, 0.6957280001724162, 0.6958432338827358, 0.6960023065289213, 0.6960505427887894, 0.6961930430561075, 0.6964094843425619, 0.6964652327220255, 0.6966813731216281, 0.6967657623610637, 0.6968132249037837, 0.6967451842025046, 0.6967467076228651, 0.6968151229760088, 0.6968750974601067, 0.6969656534939728, 0.6968722118162736, 0.6956799857222019, 0.6952770910974673, 0.6953897552052349, 0.6954815254227094, 0.6957280001724162, 0.6958432338827358, 0.6960023065289213, 0.6960505427887894, 0.6961930430561075, 0.6964094843425619,

In [40]:
#do some exploration of the data
merge_data = pd.read_csv("../output/merge_data.csv")

merge_data.head()

,click_mode,pid,sid,o1,o2,d1,d2,weekday,hour,time_index,...,similarity_from_profile_avail_8,similarity_from_profile_avail_9,similarity_from_profile_avail_10,similarity_from_profile_avail_11,sel_mode_0,sel_mode_1,sel_mode_2,sel_mode_3,sel_mode_4,sel_mode_5
0,9.0,-2.0,3000821,116.29,39.97,116.32,39.96,4,17,107,...,207.621781,972.708140,730.448868,373.138440,5.017862,2.326981,4.050950,-0.627475,-1.392469,-0.527085
1,1.0,210736.0,3085857,116.39,39.84,116.33,39.79,4,10,65,...,-0.003471,2.277442,-0.002394,-0.001754,27.163893,5.873249,3.728047,8.379609,3.202907,2.856562
2,9.0,-2.0,2944522,116.31,39.93,116.27,40.00,5,10,63,...,207.621781,972.708140,730.448868,373.138440,3.391185,0.748166,-0.666171,-2.476137,2.189719,0.655156
3,1.0,202427.0,559931,116.27,39.88,116.39,39.90,4,14,89,...,4.634204,2.119706,8.171309,8.088478,1.716232,0.029915,-0.376806,-0.290780,-0.131520,-0.795980
4,7.0,172251.0,2819352,116.34,39.96,116.37,39.86,1,11,70,...,0.021265,-0.019453,0.016131,0.012976,642.427592,-22.190209,-0.156663,-0.017854,-0.549339,0.106280


In [88]:
train = merge_data.query('click_mode !=-1')
test = merge_data.query('click_mode ==-1')

train_pid = set(train['pid'].values)
test_pid = set(test['pid'].values)

In [93]:
print(len(train_pid))
print(len(test_pid))
print(len(train_pid&test_pid))
print(len(train_pid|test_pid))

46192
13300
9325
50167


In [94]:
for col in ['profile_click_mode_embedding_{}'.format(i) for i in range(10)]:
    print(col, merge_data[col].isnull().sum())

profile_click_mode_embedding_0 7886
profile_click_mode_embedding_1 7886
profile_click_mode_embedding_2 7886
profile_click_mode_embedding_3 7886
profile_click_mode_embedding_4 7886
profile_click_mode_embedding_5 7886
profile_click_mode_embedding_6 7886
profile_click_mode_embedding_7 7886
profile_click_mode_embedding_8 7886
profile_click_mode_embedding_9 7886


In [41]:
merge_data.columns

Index(['click_mode', 'pid', 'sid', 'o1', 'o2', 'd1', 'd2', 'weekday', 'hour',
       'time_index',
       ...
       'similarity_from_profile_avail_8', 'similarity_from_profile_avail_9',
       'similarity_from_profile_avail_10', 'similarity_from_profile_avail_11',
       'sel_mode_0', 'sel_mode_1', 'sel_mode_2', 'sel_mode_3', 'sel_mode_4',
       'sel_mode_5'],
      dtype='object', length=151)

In [42]:
merge_data['pid']=merge_data['pid'].fillna(-1)
tmp = (merge_data.query('click_mode !=-1')
                 .groupby(['pid','click_mode']).size()
                 .reset_index()
                 )
tmp.columns = ['pid',  'click_mode', 'frequency']

In [43]:
tmp.describe()

,pid,click_mode,frequency
count,2961.00000,2961.000000,2961.000000
mean,156494.58899,4.158055,1.688956
std,35276.29769,3.162386,10.454996
min,-2.00000,0.000000,1.000000
25%,126565.00000,2.000000,1.000000
50%,159042.00000,3.000000,1.000000
75%,186221.00000,7.000000,1.000000
max,216973.00000,11.000000,375.000000


In [44]:
tmp.head(20)

,pid,click_mode,frequency
0,-2.0,0.0,171
1,-2.0,1.0,220
2,-2.0,2.0,375
3,-2.0,3.0,109
4,-2.0,4.0,41
5,-2.0,5.0,183
6,-2.0,6.0,38
7,-2.0,7.0,188
8,-2.0,8.0,10
9,-2.0,9.0,135


In [50]:
tmp_wide = pd.pivot_table(tmp, index = 'pid', columns = 'click_mode').fillna(0)
tmp_wide.head(20)

frequency                                                       \
click_mode      0.0    1.0    2.0    3.0   4.0    5.0   6.0    7.0   8.0    
pid                                                                         
-2.0           171.0  220.0  375.0  109.0  41.0  183.0  38.0  188.0  10.0   
-1.0            36.0    0.0    0.0    0.0   0.0    0.0   0.0    0.0   0.0   
 100011.0        2.0    1.0    0.0    1.0   1.0    0.0   0.0    1.0   0.0   
 100051.0        1.0    0.0    0.0    0.0   0.0    0.0   0.0    0.0   0.0   
 100071.0        0.0    0.0    0.0    0.0   0.0    0.0   0.0    1.0   0.0   
 100072.0        0.0    0.0    1.0    0.0   0.0    0.0   0.0    0.0   0.0   
 100081.0        0.0    0.0    0.0    0.0   0.0    0.0   0.0    0.0   0.0   
 100148.0        0.0    0.0    1.0    0.0   0.0    0.0   0.0    0.0   0.0   
 100282.0        0.0    0.0    0.0    1.0   0.0    0.0   0.0    0.0   0.0   
 100332.0        0.0    0.0    0.0    0.0   0.0    0.0   0.0    0.0   0.0   
 100357.0        0.0    1.0    0.0    0.0   0.0    0.0   0.0    0.0   0.0   
 100408.0        0.0    0.0    0.0    0.0   0.0    0.0   0.0    1.0   0.0   
 100414.0        0.0    2.0    0.0    0.0   0.0    1.0   0.0    2.0   0.0   
 100441.0        1.0    0.0    0.0    0.0   0.0    0.0   0.0    0.0   0.0   
 100471.0        0.0    0.0    1.0    0.0   0.0    0.0   0.0    0.0   0.0   
 100477.0        0.0    0.0    1.0    0.0   0.0    0.0   0.0    0.0   0.0   
 100507.0        0.0    0.0    1.0    0.0   0.0    0.0   0.0    0.0   0.0   
 100625.0        0.0    0.0    1.0    0.0   0.0    0.0   1.0    0.0   0.0   
 100686.0        0.0    0.0    0.0    0.0   1.0    0.0   0.0    0.0   0.0   
 100754.0        0.0    0.0    0.0    0.0   0.0    0.0   0.0    1.0   0.0   

                               
click_mode   9.0   10.0  11.0  
pid                            
-2.0        135.0  62.0  20.0  
-1.0          0.0   0.0   0.0  
 100011.0     0.0   0.0   0.0  
 100051.0     0.0   0.0   0.0  
 100071.0     0.0   0.0   0.0  
 100072.0     0.0   0.0   0.0  
 100081.0     1.0   0.0   0.0  
 100148.0     0.0   0.0   0.0  
 100282.0     0.0   0.0   0.0  
 100332.0     1.0   0.0   0.0  
 100357.0     0.0   0.0   0.0  
 100408.0     0.0   0.0   0.0  
 100414.0     1.0   0.0   0.0  
 100441.0     0.0   0.0   0.0  
 100471.0     0.0   0.0   0.0  
 100477.0     0.0   0.0   0.0  
 100507.0     0.0   0.0   0.0  
 100625.0     0.0   0.0   0.0  
 100686.0     0.0   0.0   0.0  
 100754.0     0.0   0.0   0.0

In [51]:
for col in tmp_wide.columns[:-1]:
    print(col, '\n')
    print(tmp_wide[col])
    

('frequency', 0.0) 

pid
-2.0         171.0
-1.0          36.0
 100011.0      2.0
 100051.0      1.0
 100071.0      0.0
 100072.0      0.0
 100081.0      0.0
 100148.0      0.0
 100282.0      0.0
 100332.0      0.0
 100357.0      0.0
 100408.0      0.0
 100414.0      0.0
 100441.0      1.0
 100471.0      0.0
 100477.0      0.0
 100507.0      0.0
 100625.0      0.0
 100686.0      0.0
 100754.0      0.0
 100761.0      0.0
 100772.0      0.0
 100783.0      0.0
 100788.0      1.0
 100789.0      0.0
 100802.0      0.0
 100811.0      0.0
 100893.0      1.0
 100966.0      0.0
 101051.0      0.0
             ...  
 215580.0      0.0
 215644.0      0.0
 215647.0      0.0
 215677.0      0.0
 215715.0      1.0
 215797.0      0.0
 215809.0      0.0
 215847.0      0.0
 215865.0      0.0
 215874.0      0.0
 215912.0      0.0
 215932.0      0.0
 216049.0      0.0
 216066.0      0.0
 216115.0      0.0
 216127.0      0.0
 216300.0      0.0
 216420.0      0.0
 216421.0      0.0
 216512.0      0.0
 21655

In [79]:
tmp_wide.columns.values

array([('frequency', 0.0), ('frequency', 1.0), ('frequency', 2.0),
       ('frequency', 3.0), ('frequency', 4.0), ('frequency', 5.0),
       ('frequency', 6.0), ('frequency', 7.0), ('frequency', 8.0),
       ('frequency', 9.0), ('frequency', 10.0), ('frequency', 11.0)],
      dtype=object)

In [2]:
profile_data = pd.read_csv('../data/profiles.csv')
profile_na = np.zeros(67)
profile_na[0] = -1
profile_na = pd.DataFrame(profile_na.reshape(1, -1))
profile_na.columns = profile_data.columns
profile_data = profile_data.append(profile_na)

In [6]:
-1 in(profile_data['pid'].values)

True

In [7]:
profile_data1 = pd.read_csv('../output/basic_profile_data.csv')
-1 in(profile_data1['pid'].values)

True